<a href="https://colab.research.google.com/github/Okocha76/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS10_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

## Engineer at least two new features.

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df.describe(exclude='number')

,created,description,display_address,street_address,interest_level
count,48817,47392,48684,48807,48817
unique,48148,37853,8674,15135,3
top,2016-06-05 01:32:36,,Broadway,3333 Broadway,low
freq,3,1627,435,174,33946


In [5]:
df.describe(include='number')

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000
mean,1.201794,1.537149,40.750760,-73.972760,3579.585247,0.524838,0.478276,0.478276,0.447631,0.424852,0.415081,0.367085,0.052769,0.268452,0.185653,0.175902,0.132761,0.138394,0.102833,0.087203,0.060471,0.055206,0.051908,0.046193,0.043305,0.042711,0.039331,0.027224,0.026241
std,0.470711,1.106087,0.038954,0.028883,1762.430772,0.499388,0.499533,0.499533,0.497255,0.494326,0.492741,0.482015,0.223573,0.443158,0.388830,0.380741,0.339320,0.345317,0.303744,0.282136,0.238359,0.228385,0.221844,0.209905,0.203544,0.202206,0.194382,0.162738,0.159852
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728300,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978000,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
col_list = ['elevator',
            'cats_allowed',
            'hardwood_floors',
            'dogs_allowed',
            'doorman',
            'dishwasher',
            'no_fee',
            'laundry_in_building',
            'fitness_center',
            'pre-war',
            'laundry_in_unit',
            'roof_deck',
            'outdoor_space',
            'dining_room',
            'high_speed_internet',
            'balcony',
            'swimming_pool',
            'new_construction',
            'terrace',
            'exclusive',
            'loft',
            'garden_patio',
            'wheelchair_access',
            'common_outdoor_space']

In [0]:
df['total_perks'] = df[col_list].sum(axis=1)

In [0]:
df['bed_2_bath'] = df['bedrooms'] / df['bathrooms']
# If zero bedrooms, bed_2_bath is infinite.
df['bed_2_bath'] = df['bed_2_bath'].replace(np.inf, np.nan)
# And if zero bathrooms, bed_2_bath is also NaN
df['bed_2_bath'].fillna(0, inplace = True)

## Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [0]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['month'] = df['created'].dt.month

In [0]:
train = df[df['month'] < 6]
test = df[df['month'] >= 6]

In [11]:
train.shape, test.shape

((31844, 37), (16973, 37))

## Fit a linear regression model with at least two features

In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
model = LinearRegression()

In [14]:
features = ['bathrooms','bedrooms','latitude','longitude', 'total_perks', 'bed_2_bath']
target = 'price'
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]
print(f'Linear Regression, dependent on: {features}')

Linear Regression, dependent on: ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'total_perks', 'bed_2_bath']


In [15]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Get the model's coefficients and intercept.

In [16]:
# model.intercept_, model.coef_
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept -1174828.4757640443
bathrooms       1778.874195
bedrooms         556.028941
latitude        1809.055829
longitude     -14892.707021
total_perks       46.291507
bed_2_bath      -145.001180


## Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [17]:
y_pred = model.predict(X_train)

rmse = mean_squared_error(y_train, y_pred)**(.5)
print(f'Train RMSE: ${rmse:,.2f}')

mae = mean_absolute_error(y_train, y_pred)
print(f'Train MAE: ${mae:,.2f}')

R2 = model.score(X_train, y_train)
print(f'Train R^2: {R2:,.4f}')

Train RMSE: $1,135.69
Train MAE: $730.00
Train R^2: 0.5846


In [18]:
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred)**(.5)
print(f'Test RMSE: ${rmse:,.2f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE: ${mae:,.2f}')

R2 = model.score(X_test, y_test)
print(f'Test R^2: {R2:,.4f}')

Test RMSE: $1,121.05
Test MAE: $733.85
Test R^2: 0.5956


## Statsmodels

Given my β-hat background, I also wanted to try out statsmodels. As expected, outcomes are almost identical. The small differences with scikit-learn could result from the different implementations of linear regression, as pointed out on https://stats.stackexchange.com/a/146809.

In [0]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [20]:
# Fit regression model on train data
results = smf.ols('price ~ bedrooms + bathrooms + latitude + longitude + total_perks + bed_2_bath', data=train).fit()

rmse = results.mse_resid**(.5)
print(f'Train RMSE: ${rmse:,.2f}')

mae = mean_absolute_error(y_train, results.fittedvalues)
print(f'Train MAE: ${mae:,.2f}')

R2 = results.rsquared
print(f'Train R^2: {R2:,.4f}')

print(results.summary())

Train RMSE: $1,135.82
Train MAE: $730.00
Train R^2: 0.5846
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     7468.
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:19:30   Log-Likelihood:            -2.6921e+05
No. Observations:               31844   AIC:                         5.384e+05
Df Residuals:                   31837   BIC:                         5.385e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [21]:
# Fit regression model on test data
results = smf.ols('price ~ bedrooms + bathrooms + latitude + longitude + total_perks + bed_2_bath', data=test).fit()

rmse = results.mse_resid**(.5)
print(f'Test RMSE: ${rmse:,.2f}')

mae = mean_absolute_error(y_test, results.fittedvalues)
print(f'Test MAE: ${mae:,.2f}')

R2 = results.rsquared
print(f'Test R^2: {R2:,.4f}')

# Inspect the results
print(results.summary())

Test RMSE: $1,120.63
Test MAE: $739.01
Test R^2: 0.5961
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.596
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     4173.
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:19:30   Log-Likelihood:            -1.4326e+05
No. Observations:               16973   AIC:                         2.865e+05
Df Residuals:                   16966   BIC:                         2.866e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------